<a href="https://colab.research.google.com/github/WahyuKhairi06/TextMining_WahyuKhairi_2311531009/blob/main/Tugas%204/K_means_Text_Mining_Wahyu_Khairi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TUGAS 4 TEKS MINING**

**Import Library**

In [16]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

**Dataset**

In [17]:
# Dataset dokumen
docs = [
    "makan ayam enak",       # D1
    "makan nasi",            # D2
    "ayam goreng enak",      # D3
    "python belajar",        # D4
    "machine python",        # D5
    "belajar python machine" # D6
]

# Kosakata
vocab = ["makan", "ayam", "enak", "nasi", "goreng", "python", "belajar", "machine"]

**BoW**

In [18]:
# Representasi Bag-of-Words biner
X = []
for d in docs:
    X.append([1 if w in d.split() else 0 for w in vocab])

X = np.array(X)
df = pd.DataFrame(X, columns=vocab, index=[f"D{i+1}" for i in range(len(docs))])
print("=== Representasi BoW Binary ===")
print(df)

=== Representasi BoW Binary ===
    makan  ayam  enak  nasi  goreng  python  belajar  machine
D1      1     1     1     0       0       0        0        0
D2      1     0     0     1       0       0        0        0
D3      0     1     1     0       1       0        0        0
D4      0     0     0     0       0       1        1        0
D5      0     0     0     0       0       1        0        1
D6      0     0     0     0       0       1        1        1


**Centroid** awal

In [19]:
# Inisialisasi centroid awal: C1 = D1, C2 = D4
C1 = X[0]  # D1
C2 = X[3]  # D4
centroids = np.array([C1, C2], dtype=float)

**Kmeans 2 iterasi**

In [20]:
#K-Means (2 Iterasi)

for iteration in range(2):
    print(f"\n=== Iterasi {iteration + 1} ===")

    # Hitung jarak Euclidean setiap dokumen ke tiap centroid
    distances = np.zeros((len(X), 2))
    for i, x in enumerate(X):
        for j, c in enumerate(centroids):
            distances[i, j] = np.linalg.norm(x - c)

    df_dist = pd.DataFrame(distances, columns=["C1", "C2"], index=df.index)
    print("\nJarak ke Centroid:")
    print(df_dist)

    # Assign cluster berdasarkan jarak minimum
    cluster_assign = np.argmin(distances, axis=1)
    df_cluster = pd.DataFrame({
        "Dokumen": df.index,
        "Cluster": ["C1" if c == 0 else "C2" for c in cluster_assign]
    })
    print("\nAssignment Cluster:")
    print(df_cluster)

    # Update centroid (rata-rata anggota tiap cluster)
    new_centroids = []
    for k in range(2):
        members = X[cluster_assign == k]
        new_c = members.mean(axis=0)
        new_centroids.append(new_c)
    centroids = np.array(new_centroids)
    print("\nCentroid Baru:")
    print(pd.DataFrame(centroids, columns=vocab, index=["C1", "C2"]))



=== Iterasi 1 ===

Jarak ke Centroid:
          C1        C2
D1  0.000000  2.236068
D2  1.732051  2.000000
D3  1.414214  2.236068
D4  2.236068  0.000000
D5  2.236068  1.414214
D6  2.449490  1.000000

Assignment Cluster:
  Dokumen Cluster
0      D1      C1
1      D2      C1
2      D3      C1
3      D4      C2
4      D5      C2
5      D6      C2

Centroid Baru:
       makan      ayam      enak      nasi    goreng  python   belajar  \
C1  0.666667  0.666667  0.666667  0.333333  0.333333     0.0  0.000000   
C2  0.000000  0.000000  0.000000  0.000000  0.000000     1.0  0.666667   

     machine  
C1  0.000000  
C2  0.666667  

=== Iterasi 2 ===

Jarak ke Centroid:
          C1        C2
D1  0.745356  2.211083
D2  1.247219  1.972027
D3  1.105542  2.211083
D4  1.885618  0.745356
D5  1.885618  0.745356
D6  2.134375  0.471405

Assignment Cluster:
  Dokumen Cluster
0      D1      C1
1      D2      C1
2      D3      C1
3      D4      C2
4      D5      C2
5      D6      C2

Centroid Baru:
      

**Kmeans Sklearn**

In [21]:
# KMeans sklearn

kmeans = KMeans(n_clusters=2, random_state=0, n_init='auto').fit(X)
labels = kmeans.labels_

print("\n=== Hasil KMeans sklearn ===")
for i, label in enumerate(labels):
    print(f"{df.index[i]} → Cluster {label+1}")



=== Hasil KMeans sklearn ===
D1 → Cluster 2
D2 → Cluster 1
D3 → Cluster 2
D4 → Cluster 1
D5 → Cluster 1
D6 → Cluster 1


**hasil**

In [22]:
# Interpretasi hasil
print("\n=== Interpretasi ===")
print("Cluster 1 (manual): kemungkinan topik makanan")
print("Cluster 2 (manual): kemungkinan topik pemrograman")

print("\nKata dominan pada tiap centroid manual:")
for i, c in enumerate(centroids):
    dominant = [vocab[j] for j, val in enumerate(c) if val >= 0.5]
    print(f"C{i+1}: {dominant}")


=== Interpretasi ===
Cluster 1 (manual): kemungkinan topik makanan
Cluster 2 (manual): kemungkinan topik pemrograman

Kata dominan pada tiap centroid manual:
C1: ['makan', 'ayam', 'enak']
C2: ['python', 'belajar', 'machine']
